In [ ]:
import pickle
import numpy as np

In [ ]:
data_file = "results_5_10_25_individuals.pickle"

In [ ]:
def convert_to_list_int(value):
    return sorted([int(v) for v in value.split(" ")[:-1]])

In [ ]:
convert_dict = {'Best_idx':int,'Best_fitness':float,'Worst_idx':int,'Worst_fitness':float,'Avg_fitness':float,'Best_genome':str}
with open(data_file, 'rb') as handle:
    data = pickle.load(handle)
    for run in data:
        for idx, trial in enumerate(run[1]):
            run[1][idx] = trial.astype(convert_dict)
            run[1][idx]["Best_genome"] = run[1][idx]["Best_genome"].apply(convert_to_list_int)
            

In [ ]:
def get_best_try_of_run(run):
    best_value = None
    best_try = None
    for trial in run[1]:
        new_value = trial.iloc[-1,:]["Best_fitness"]
        if best_try is None or new_value < best_value:
            best_try = trial
            best_value = new_value
    return run[0], best_try

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
fig, ax = plt.subplots(figsize=[8, 3])
for run in data:
    run_id = run[0]
    if "0.0 0.0" in run_id:
        for trial in run[1]:
            ax.plot(trial["Best_fitness"], label=run_id)
#plt.legend()
#plt.show()
plt.title("All the runs with prob_crossover=0.0 and prob_mutation=0.0")
plt.ylabel("Fitness")
plt.xlabel("Iterations")
plt.tight_layout()
plt.savefig("docs/assets/no_cross_no_mut.pdf", format="pdf")

In [ ]:
fig, ax = plt.subplots(figsize=[8, 3])
for run in data:
    run_id = run[0]
    if "1.0 1.0" in run_id:
        for trial in run[1]:
            if "2 1.0 1.0" in run_id:
                ax.plot(trial["Best_fitness"], "r-", label=run_id)
                continue
            if "3 1.0 1.0" in run_id:
                ax.plot(trial["Best_fitness"], "b-", label=run_id)
                continue
            if "4 1.0 1.0" in run_id:
                ax.plot(trial["Best_fitness"], "g-", label=run_id)
                continue
#plt.legend()
#plt.show()
plt.title("All the runs with prob_crossover=1.0 and prob_mutation=1.0")
plt.ylabel("Fitness")
plt.xlabel("Iterations")
r_patch = mpatches.Patch(color='red', label='2 hubs')
b_patch = mpatches.Patch(color='blue', label='3 hubs')
g_patch = mpatches.Patch(color='green', label='4 hubs')
plt.legend(handles=[r_patch, b_patch, g_patch])
plt.tight_layout()
plt.savefig("docs/assets/max_cross_mut.pdf", format="pdf")

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=[30, 6], sharey=True)
for run in data:
    run_id = run[0]
    for trial in run[1]:
        for v in range(0,12,2):
            if f"4 {v/10} {v/10}" in run_id:
                ax[int(v/2)].set_title(f"Probability crossover={v/10} mutation={v/10}")
                ax[int(v/2)].plot(trial["Best_fitness"], c=[v/10,0,0], label=run_id)
                continue
plt.xlabel("Iterations")
plt.tight_layout()
plt.savefig("docs/assets/sweep_values_get_best_fitness.pdf", format="pdf")

In [ ]:
data_file = "results_25_individuals.pickle"

In [ ]:
convert_dict = {'Best_idx':int,'Best_fitness':float,'Worst_idx':int,'Worst_fitness':float,'Avg_fitness':float,'Best_genome':str}
with open(data_file, 'rb') as handle:
    data = pickle.load(handle)
    for run in data:
        for idx, trial in enumerate(run[1]):
            run[1][idx] = trial.astype(convert_dict)
            run[1][idx]["Best_genome"] = run[1][idx]["Best_genome"].apply(convert_to_list_int)

In [ ]:
fig, ax = plt.subplots(figsize=[10, 7])
for run in data:
    run_id = run[0]
    for trial in run[1]:
        if "25 4 0.2" in run_id:
            ax.plot(trial["Avg_fitness"], label=run_id)
#plt.legend()
plt.tight_layout()
plt.savefig("docs/assets/all_runs_02.pdf", format="pdf")

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=[30, 6], sharey=True)
for run in data:
    run_id, trial = get_best_try_of_run(run)
    for j in range(0,12,2):
        for v in range(0,12,2):
            if f"4 {j/10} {v/10}" in run_id:
                ax[int(v/2)].set_title(f"Probability crossover=[0,1] mutation={v/10}")
                ax[int(v/2)].plot(trial["Avg_fitness"], c=[j/10,0,0], label=run_id)
plt.xlabel("Iterations")
leg = []
for c in range(0,12,2):
    leg.append(mpatches.Patch(color=[0,0,c/10], label=f'{c/10} crossover'))
    ax[int(c/2)].grid()
plt.legend(handles=leg)
plt.tight_layout()
plt.savefig("docs/assets/sweep_values_get_avg_fitness.pdf", format="pdf")

# When is the optimal solution achieved

In [ ]:
import pandas as pd

In [ ]:
optimal_known_solutions = [[3, 19], [7, 19, 20], [3, 6, 7, 19]]

In [ ]:
def is_optimal_solution(trial):
    genomes_trial = np.array(trial["Best_genome"].to_list())
    for know_solution in optimal_known_solutions:
        if genomes_trial.shape[1] != np.array(know_solution).shape[0]:
            continue
        iteration_solution = np.where(np.all(genomes_trial == np.array(know_solution), axis=1))[0]
        if len(iteration_solution):
            return iteration_solution[0]
    return None

fig, ax = plt.subplots(2, 1, figsize=[15, 8], sharex=True)
results = []
for run in data:
    for trial in run[1]:
        idx = is_optimal_solution(trial)
        if idx is not None:
            results.append([run[0], idx])
        else:
            results.append([run[0], None])
results = pd.DataFrame(results, columns=["run", "n_iter"])
mean_run = results.groupby("run").mean()
count_run = results.groupby("run").count()

ax[0].bar(list(range(len(mean_run.index))), mean_run["n_iter"].to_numpy(),tick_label=mean_run.index, color="red")
ax[0].set_xticklabels(mean_run.index, rotation=85)
ax[0].set_ylabel("Avg iterations to optimal solution (s)")
ax[0].grid()

ax[1].bar(list(range(len(count_run.index))), count_run["n_iter"].to_numpy()*100/30,tick_label=count_run.index, color="red")
ax[1].set_xticklabels(count_run.index, rotation=85)
ax[1].set_ylabel("% runs optimal solution (%)")
ax[1].grid()
plt.xlabel("Run parameters")
plt.tight_layout()
plt.savefig("docs/assets/iters_to_optimal.pdf", format="pdf")

In [ ]:
convert_dict = {'Best_idx':int,'Best_fitness':float,'Worst_idx':int,'Worst_fitness':float,'Avg_fitness':float,'Best_genome':str}
with open("results_25_0_to_02_individuals.pickle", 'rb') as handle:
    data = pickle.load(handle)
    for run in data:
        for idx, trial in enumerate(run[1]):
            run[1][idx] = trial.astype(convert_dict)
            run[1][idx]["Best_genome"] = run[1][idx]["Best_genome"].apply(convert_to_list_int)

In [ ]:
def is_optimal_solution(trial):
    genomes_trial = np.array(trial["Best_genome"].to_list())
    for know_solution in optimal_known_solutions:
        if genomes_trial.shape[1] != np.array(know_solution).shape[0]:
            continue
        iteration_solution = np.where(np.all(genomes_trial == np.array(know_solution), axis=1))[0]
        if len(iteration_solution):
            return iteration_solution[0]
    return None

fig, ax = plt.subplots(2, 1, figsize=[15, 8], sharex=True)
results = []
for run in data:
    for trial in run[1]:
        idx = is_optimal_solution(trial)
        if idx is not None:
            results.append([run[0], idx])
        else:
            results.append([run[0], None])
results = pd.DataFrame(results, columns=["run", "n_iter"])
mean_run = results.groupby("run").mean()
count_run = results.groupby("run").count()

ax[0].bar(list(range(len(mean_run.index))), mean_run["n_iter"].to_numpy(),tick_label=mean_run.index, color="red")
ax[0].set_xticklabels(mean_run.index, rotation=85)
ax[0].set_ylabel("Avg iterations to optimal solution (s)")
ax[0].grid()

ax[1].bar(list(range(len(count_run.index))), count_run["n_iter"].to_numpy()*100/30,tick_label=count_run.index, color="red")
ax[1].set_xticklabels(count_run.index, rotation=85)
ax[1].set_ylabel("% runs optimal solution (%)")
ax[1].grid()
plt.xlabel("Run parameters")
plt.tight_layout()
plt.savefig("docs/assets/iters_to_optimal_02.pdf", format="pdf")